In [131]:
'''
Scraping date from baseball-reference.com
This file is specifically for getting information that will be used to predict the probability of a batter hitting a 
single.
Notes:
    -Specific to the blue jays
    -Specific to the year
-Matthew Sims
'''

'\nScraping date from baseball-reference.com\nThis file is specifically for getting information that will be used to predict the probability of a batter hitting a \nsingle.\nNotes:\n    -Specific to the blue jays\n    -Specific to the year\n-Matthew Sims\n'

In [351]:
#list of libraries being imported
from bs4 import BeautifulSoup
import urllib
import collections

In [352]:
#function used to convert the date from the form (year, month_name day_number) to (yyyymmdd)
def dateConversion(oldDate):
    newDate = ""
    month = ""
    day_num = ""
    dateSystemDict = {"Jan": "01",
                      "Feb": "02",
                      "Mar": "03",
                      "Apr": "04",
                      "May": "05",
                      "Jun": "06",
                      "Jul": "07",
                      "Aug": "08",
                      "Sep": "09",
                      "Oct": "10",
                      "Nov": "11",
                      "Dec": "12"}
    oldDate = oldDate.split(' ') #the default is space, but it is used here to be explicit
    splitSize = len(oldDate)
    month = dateSystemDict[oldDate[splitSize-2]]
    day_num = oldDate[splitSize-1]
    if (len(day_num) == 1):
        day_num = "0" + day_num
    elif (len(day_num) < 1 or len(day_num) > 2):
        print "error getting date from schedule table"
    newDate = year + month + day_num
    return newDate
        

In [353]:
#function to change the team abbreviation if needed
#the team abrs used in the table do not match up with the ones used in the url for boxscores
def teamAbrConversion(teamAbr):
    teamAbrDict = {"NYY":"NYA",
                   "TBR":"TBA",
                   "WSN":"WAS",
                   "NYM":"NYN",
                   "CHW":"CHA",
                   "KCR":"KCA",
                   "LAA":"ANA"}
    if (teamAbrDict.has_key(teamAbr)):
        return teamAbrDict[teamAbr]
    return teamAbr

In [354]:
#function to loop through a team's schedule a grab all the url needed to request boxscores
#hard-coded Toronto as the home team
def getListofBoxscoreURLs(schedule_table):
    listOfUrls = []
    #get a list of all the rows in the schedule table
    team_schedule_rows = schedule_table.find_all('tr')
    team_schedule_rows.pop(0)
    for a_row in team_schedule_rows:
        try:
            columns = a_row.find_all("td")
            base_url = "http://www.baseball-reference.com/boxes/"
            end_url = "0.shtml"
            oldTeamAbr = a_row.find(id="Click for standings on this date")
            date = dateConversion(a_row.find(title="Click for standings on this date").get_text())
            #find the home team by first looking in the row for an @ symbol
            homeTeamAbr = ""
            if ("@" in columns[5].get_text()):
                homeTeamAbr = teamAbrConversion(columns[6].get_text())
            else:
                homeTeamAbr = teamAbrConversion(abr1[team_index])
            url = base_url + homeTeamAbr + '/' + homeTeamAbr + date + end_url
            listOfUrls.append(url.encode('utf-8'))
        except:
            continue
    return listOfUrls

In [355]:
testURL = "http://www.baseball-reference.com/boxes/NYA/NYA201504060.shtml"

In [356]:
boxscore_page = urllib.urlopen(testURL).read()
boxscore_page = BeautifulSoup(boxscore_page)

In [357]:
#function to scan through the play-by-play finding single in the description to get the single stats for each plater
def getSingles(page):
    singlesList = []
    play_by_play_table = page.find(id="play_by_play")
    play_by_play_rows = play_by_play_table.find_all('tr')
    for i in range(0, len(play_by_play_rows)):
        try:
            columns = play_by_play_rows[i].find_all('td')
            event = columns[len(columns)-1].get_text()
            if ('Single' in event):
                nameFound = False
                lastName = columns[7].get_text().split()
                lastName = lastName[len(lastName)-1]
                singleDict = {"lastName":lastName.encode('utf-8'), "singleCount": 1}
                for j in range(0, len(singlesList)):
                    if (singlesList[j]["lastName"] == lastName):
                        #print lastName
                        nameFound = True
                        singlesList[j]["singleCount"] += 1
                if (not nameFound):
                    singlesList.append(singleDict)
        except:
            continue
    return singlesList
        

In [358]:
#function to return the opposing pitcher on the day
#getPitcher from the play by play table
def getPitcher(page, batter):
    play_by_play_table = page.find(id="play_by_play")
    play_by_play_rows = play_by_play_table.find_all('tr')
    for i in range(0, len(play_by_play_rows)):
        try:
            columns = play_by_play_rows[i].find_all('td')
            b_lastName = columns[7].get_text().split()
            b_lastName = b_lastName[len(b_lastName)-1]
            if (b_lastName in batter):
                p_lastName = columns[8].get_text().split()
                p_lastName = p_lastName[len(p_lastName)-1]
                return p_lastName.encode('utf-8')
        except:
            continue
    return "failed to find pitcher"
        

In [359]:
print getPitcher(boxscore_page, 'Jacoby Ellsbury')

Hutchison


In [368]:
#hard-coded for the Toronto Blue Jays
#column accesses are hard_coded by index
#for boxscoreURL in urlList:
    #opening the boxscore page 
def getTeamBattingStats(page, listOfSingles):
    boxscore_batting_table = page.find(id=currentTeam[team_index] + "batting")
    batter_rows = boxscore_batting_table.find_all('tr')
    batter_rows.pop(0)
    gameBattingStats = []
    for batter_info in batter_rows:
        foundName = False
        record = {"name":"",
                  "PA": 0,
                  "OBP": 0,
                  "SLG": 0,
                  "singles": 0,
                  "pitcher_name": "",
                  "pitcher_OBP": 0,
                  "defensive_efficiency": 0,
                  "team": currentTeam[team_index]}
        attributes = batter_info.find_all('td')
        name = attributes[0].get_text().strip().split(' ')
        name.pop(len(name)-1)
        record["name"] = ' '.join(name).strip().encode('utf-8')
        record["PA"] = (attributes[7].get_text()).encode('utf-8')
        record["OBP"] = (attributes[9].get_text()).encode('utf-8')
        record["SLG"] = (attributes[10].get_text()).encode('utf-8')
        record["pitcher_name"] = getPitcher(page, record["name"])
        if (record["pitcher_name"] != "failed to find pitcher" and record["PA"] != ""):
            for single in listOfSingles:
                if (single["lastName"] in record["name"]):
                    record["singles"] = single["singleCount"]
                    #print "addedSingle"
                    break
            gameBattingStats.append(record)
    return gameBattingStats
    


In [369]:
boxscore_page = urllib.urlopen("http://www.baseball-reference.com/boxes/NYA/NYA201504060.shtml").read()
boxscore_page = BeautifulSoup(boxscore_page)
singless =  getSingles(boxscore_page)
print singless

[{'lastName': 'Navarro', 'singleCount': 1}, {'lastName': 'Pillar', 'singleCount': 2}, {'lastName': 'Martin', 'singleCount': 1}, {'lastName': 'McCann', 'singleCount': 1}, {'lastName': 'Rodriguez', 'singleCount': 1}]


In [397]:
#function to get all urls from a team's schedule then get all the batter information
def getAllTeamBattingStats(t_schedule):
    #variable to store all data records to be inputted into excel
    data = []
    #get all the urls
    urlList = getListofBoxscoreURLs(t_schedule)
    #open each boxscore page to grab info
    for url in urlList:
        try:
            boxscore_page = urllib.urlopen(url).read()
            boxscore_page = BeautifulSoup(boxscore_page)
            singlesList = getSingles(boxscore_page)
            data = data + getTeamBattingStats(boxscore_page, singlesList)
            print url
        except:
            print "fails:" + url
    return data

In [ ]:
#global variables
currentTeam = ["TorontoBlueJays", "NewYorkYankees", "BostonRedSox", "BaltimoreOrioles", "TampaBayRays", 
               "DetroitTigers", "NewYorkMets", "KansasCityRoyals", "WashingtonNationals", "TexasRangers",
               "OaklandAthletics", "MiamiMarlins", "PhiladelphiaPhillies", "HoustonAstros", "SeattleMariners",
               "MinnesotaTwins", "ChicagoWhiteSox", "ClevelandIndians", "AtlantaBraves", "AnaheimAngels"]
abr1 = ["TOR", "NYY", "BOS", "BAL", "TBR", "DET", "NYM", "KCR", "WSN", "TEX", "OAK", "MIA", "PHI", "HOU", "SEA",
        "MIN", "CHW", "CLE", "ATL", "LAA"]
year = "2015"
data = []
for team_index in range(0, len(currentTeam)):
    currentTeamURL = "http://www.baseball-reference.com/teams/" + abr1[team_index] + "/2015-schedule-scores.shtml"
    #print currentTeamURL
    #opening the team schedule page 
    try:
        print currentTeam[team_index]
        team_schedule = urllib.urlopen(currentTeamURL).read()
        team_schedule = BeautifulSoup(team_schedule)
        team_schedule_table = team_schedule.find(id="team_schedule")
        data = data + getAllTeamBattingStats(team_schedule_table)
    except:
        print "fail:" + currentTeamURL

TorontoBlueJays
http://www.baseball-reference.com/boxes/NYA/NYA201504060.shtml
http://www.baseball-reference.com/boxes/NYA/NYA201504080.shtml
http://www.baseball-reference.com/boxes/NYA/NYA201504090.shtml
http://www.baseball-reference.com/boxes/BAL/BAL201504100.shtml
http://www.baseball-reference.com/boxes/BAL/BAL201504110.shtml
http://www.baseball-reference.com/boxes/BAL/BAL201504120.shtml
http://www.baseball-reference.com/boxes/TOR/TOR201504130.shtml
http://www.baseball-reference.com/boxes/TOR/TOR201504140.shtml
http://www.baseball-reference.com/boxes/TOR/TOR201504150.shtml
http://www.baseball-reference.com/boxes/TOR/TOR201504160.shtml
http://www.baseball-reference.com/boxes/TOR/TOR201504170.shtml
http://www.baseball-reference.com/boxes/TOR/TOR201504180.shtml
http://www.baseball-reference.com/boxes/TOR/TOR201504190.shtml
http://www.baseball-reference.com/boxes/TOR/TOR201504210.shtml
http://www.baseball-reference.com/boxes/TOR/TOR201504220.shtml
http://www.baseball-reference.com/boxes

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.to_excel('test.xlsx', sheet_name='sheet1', index=False)

In [ ]:
print df